In [3]:
from collections import defaultdict
import pandas as pd
import numpy as np
import sqlite3
import pdb
import os

%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
pio.templates
import seaborn as sns

Outpatient Descriptive Analysis

In [4]:
outpatient_df = pd.read_csv('idea-1/medicare-data/medicare-data/Outpatient_provdr.csv')

In [5]:
outpatient_df.describe()

,Provider Id,Provider Zip Code,Outpatient Services,Average Estimated Submitted Charges,Average Total Payments
count,43372.000000,43372.000000,43372.000000,43372.000000,43372.000000
mean,265410.921747,49041.022434,587.980955,1473.994047,265.425586
std,152729.927698,27497.899751,3280.397168,1837.530791,261.260146
min,10001.000000,1040.000000,11.000000,12.643750,7.305000
25%,140043.000000,28645.000000,42.000000,284.610498,74.627714
50%,260091.000000,46342.000000,128.000000,709.995188,139.298570
75%,390101.000000,73101.000000,347.000000,2102.121346,394.211316
max,670061.000000,99801.000000,219042.000000,32105.636360,2062.470526


In [6]:
outpatient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43372 entries, 0 to 43371
Data columns (total 11 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   APC                                         43372 non-null  object 
 1   Provider Id                                 43372 non-null  int64  
 2   Provider Name                               43372 non-null  object 
 3   Provider Street Address                     43372 non-null  object 
 4   Provider City                               43372 non-null  object 
 5   Provider State                              43372 non-null  object 
 6   Provider Zip Code                           43372 non-null  int64  
 7   Hospital Referral Region (HRR) Description  43372 non-null  object 
 8   Outpatient Services                         43372 non-null  int64  
 9   Average  Estimated Submitted Charges        43372 non-null  float64
 10  Average To

In [7]:
outpatient_df.corr()

,Provider Id,Provider Zip Code,Outpatient Services,Average Estimated Submitted Charges,Average Total Payments
Provider Id,1.000000,-0.132199,0.011141,-0.051995,-0.024966
Provider Zip Code,-0.132199,1.000000,-0.013416,0.043500,0.000122
Outpatient Services,0.011141,-0.013416,1.000000,-0.080775,-0.080738
Average Estimated Submitted Charges,-0.051995,0.043500,-0.080775,1.000000,0.768358
Average Total Payments,-0.024966,0.000122,-0.080738,0.768358,1.000000


In [8]:
#Outpatient Provider City
city_count  = outpatient_df['Provider City'].value_counts()
city_count = city_count[:10,]
fig = px.bar(x = city_count.index, y = city_count.values)
fig.update_layout(
    title='Outpatient Provider in Top 10 cities',
    template='ggplot2',
    xaxis=dict(
        title='Provider City Id',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title='Count',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
)
fig.layout.xaxis.type = 'category'
fig.show()

In [9]:
# Outpatient by State
state_count  = outpatient_df['Provider State'].value_counts()
state_count = state_count[:10,]
fig = px.histogram(x = state_count.index, y = state_count.values)
fig.update_layout(
    title='Outpatient Provider in Top 10 States',
    template='ggplot2',
    xaxis=dict(
        title='States',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title='Count',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1 
)
fig.layout.xaxis.type = 'category'
fig.show()


In [10]:
# Outpatient by Zip Code
zipcode_count  =outpatient_df['Provider Zip Code'].value_counts()
zipcode_count = zipcode_count[:10,]
fig = px.histogram(x = zipcode_count.index, y = zipcode_count.values)
fig.update_layout(
    title='Outpatient Provider in Top 10 Zip Codes',
    template='ggplot2',
    xaxis=dict(
        title='Zip Codes',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title='Count',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1 
)
fig.layout.xaxis.type = 'category'
fig.show()